This notebook will take the corpus, translate it into a gensim TaggedDocument list, and train the doc2vec model in order to extract the document vecotrs.  We will then translate that into a pandas data frame and save to a file for use in kmeans clustering.

In [ ]:
import pickle
import numpy
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
from os import path
import sys

state_gov_nm = 'CA'
year_to_process = '2020'
data_file_location = 'C:/Users/William Mitchell/Documents/QuigleyData'

#state_gov_nm = sys.argv[1]
#year_to_process = sys.argv[2]
#data_file_location = sys.argv[3]

In [ ]:
_embeddings = path.join(data_file_location, state_gov_nm + 'embedded_legdoc.df')
_winlist = path.join(data_file_location, 'test.df')

In [3]:
legcorpus = []
#win_state = []
for i in range(2009, int(year_to_process)+1):
    print(i)
    #if 1==1:
    try:
        #_WinList = path.join(data_file_location, state_gov_nm + str(i) + 'WinList.list')
        _legtext = path.join(data_file_location, state_gov_nm + str(i) + 'legtext.corpus')
        #_LegOverallData_CSV = path.join(data_file_location, state_gov_nm + str(i) + 'LegOverallData.csv')

        with open(_legtext,'rb') as f:
            legcorpus.extend(pickle.load(f))
        #print(i)
        #print(len(legcorpus))
        
        #leg_data_raw = leg_data_raw.append(pd.read_csv(_LegOverallData_CSV))
        
        #print(leg_data.shape)
        
        #with open(_WinList,'rb') as f:
        #    win_state.extend(pickle.load(f))
        
        #print(len(win_state))
    except:
        continue


#with open(_legtext,'rb') as f:
#    legcorpus = pickle.load(f)
#remove legslative stop words (if not accomplished in the legislation retrieval)
#leg_stop_words = ['rep','sen','bold','italics','sponsors']
print(len(legcorpus))
#print(legcorpus[1])

2009
2010
2011
2012
2013
2014
2015

KeyboardInterrupt: 

In [ ]:
#legcorpus = [x[] if not isinstance(x[2], list) else x for x in legcorpus]
clean_legcorpus = []
for i, b in enumerate(legcorpus):    
    if not isinstance(b[2],list):
    #    print(i, type(b[2]))
        clean_legcorpus.append([b[0],b[1],[]])
    else:
        clean_legcorpus.append(b)


In [ ]:
print(len(clean_legcorpus))
print(clean_legcorpus[0])

In [ ]:
texts = []
bill_id = []
for b in clean_legcorpus:
    texts.append(b[2])
    bill_id.append(b[0])

In [ ]:
d = {'texts': texts, 'bill_id':bill_id}
tagged_leg_df = pd.DataFrame(d)
tagged_leg_df.set_index('bill_id',inplace=True)
#tagged_leg_df = tagged_leg_df.join(labels)
#print(tagged_leg_df.head())

#tagged_leg_df['clean'] = tagged_leg_df['target'].fillna(0.0)

#int_label = []

#for i in tagged_leg_df['clean']:
#    int_label.append(int(i))

#tagged_leg_df['target_int'] = int_label

print(tagged_leg_df.head())

In [ ]:
# here we'll transform the legcorpus into a "tagged" document for use with doc2vec
# the tags will simply be the document ids.  Since we won't be using the predictor part of the operation, and only the resulting weights,
# we don't need to concern ourselves with labeling the documents
tagged_legdocs = []
for l in clean_legcorpus:
    #if i == 1:
        #print(td(l[2], [l[0]]))
    tagged_legdocs.append(td(l[2], [str(l[0])]))

In [ ]:
print(tagged_legdocs[1])

In [ ]:
vector_size = 20
embedder = d2v(tagged_legdocs, dm=1, alpha=.025,vector_size=vector_size, min_alpha=.025, min_count=0, epochs=20, workers=4)

In [ ]:
embedder.train(tagged_legdocs, total_examples=embedder.corpus_count, epochs=embedder.epochs)

In [ ]:
print(len(embedder.docvecs))
print(type(embedder.docvecs))

In [ ]:
# translate the embeddings and tags into a dataframe
doc_embedding_list = []
#for i in range(0,9):
for i in range(len(embedder.docvecs)):
    #print(embedder.docvecs.index_to_doctag(i))
    doc_tag = embedder.docvecs.index_to_doctag(i)
    doc_vec = [doc_tag]
    doc_vec.extend(embedder.docvecs[doc_tag].tolist())
    #print(doc_vec)
    #doc_vec = doc_vec.tolist()
    
    doc_embedding_list.append(doc_vec)

DFlabels = ['bill_id']

for i in range(1,vector_size + 1):
    DFlabels.append('dv' + str(i))

print(DFlabels)

doc_embed_DF = pd.DataFrame(doc_embedding_list, columns=DFlabels)

In [ ]:
#dump data out to a binary file
with open(_embeddings,'wb') as f:
    pickle.dump(doc_embed_DF,f)